In [0]:
%sql
select * from democatalog.silver.orders_json

### Access elements from the JSON object

In [0]:
%sql
select json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       json_value.items
  from democatalog.silver.orders_json

### Deduplicate Array elements

In [0]:
%sql
select json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       array_distinct(json_value.items) as items
  from democatalog.silver.orders_json

###   Expolode Arrays

In [0]:
%sql
select json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       explode(json_value.items) as item
  from democatalog.silver.orders_json

In [0]:
%sql
create  or replace temporary view tv_orders_exploded
as
select json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       explode(json_value.items) as item
  from democatalog.silver.orders_json


In [0]:
%sql
select order_id,
       order_status,
       payment_method,
       total_amount,  
       transaction_timestamp,
       customer_id,
       item.item_id,
       item.name,
       item.price,
       item.quantity,
       item.category,
       item.details.brand,
       item.details.color
 from tv_orders_exploded

### Write the transformed data into the silver schema

In [0]:
%sql
create table if not exists democatalog.silver.orders
as 
select order_id,
       order_status,
       payment_method,
       total_amount,  
       transaction_timestamp,
       customer_id,
       item.item_id,
       item.name,
       item.price,
       item.quantity,
       item.category,
       item.details.brand,
       item.details.color
 from tv_orders_exploded

In [0]:
%sql
select * from democatalog.silver.orders